In [6]:
import openai
import pandas as pd
from collections import Counter
import os

In [7]:
'''
preprocess
 1.delete all rows include 'None'
 2.combine 
'''
jobs = pd.read_excel('boss_1.xlsx')
# print(Counter(jobs['adjusted_job']))

jobs = jobs[jobs['detail'].notna()]
jobs = jobs[jobs['adjusted_job'].notna()]
jobs = jobs[jobs['job'].notna()]
jobs = jobs[jobs['经验要求'].notna()]
jobs = jobs[jobs['学历'].notna()]
jobs = jobs[jobs['工资'].notna()]

jobs.to_csv('processed_jobs.csv', index=False)

job_counts = jobs.groupby('adjusted_job').size()
jobs_merged = jobs.groupby('adjusted_job').agg({
    'detail': ' '.join,
    '工资': ' '.join,
    '学历': ' '.join,
    '经验要求': ' '.join
}).reset_index()

jobs_merged = jobs_merged.rename(columns={
    '工资': 'salary',
    '学历': 'edu_bg',
    '经验要求': 'expe',
})
print(jobs_merged.columns)

Index(['adjusted_job', 'detail', 'salary', 'edu_bg', 'expe'], dtype='object')


In [8]:
print(jobs_merged['adjusted_job'])

0           产品经理
1           人事专员
2           人力资源
3       供应链主管/经理
4           咨询顾问
5             客服
6           市场营销
7           广告设计
8     总裁/总经理/CEO
9           文科教师
10            法务
11          物流专员
12           管培生
13            编辑
14       行政助理/文员
15           运营岗
16          销售人员
17          项目管理
Name: adjusted_job, dtype: object


In [11]:
openai.api_key = "sk-nGwRkt4DhrQNpM1X01Af6b17E53b41FbB7E2B082E6C5B6D5"

# all client options can be configured just like the `OpenAI` instantiation counterpart
openai.base_url = "https://free.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

def extract_job_details(adjusted_job, detail, edu_bg, expe):
    prompt = f"""你是一位经验丰富的HR专家。请根据以下信息，为每个指标打分（0-5）,请仅返回分数，格式如下：
专业知识与技能分数, 学习能力分数, 通识性知识与技能分数, 沟通与表达能力分数, 创新能力分数, 执行能力分数, 自我管理能力分数, 职业适应能力分数, 动手操作能力分数, 承压抗挫能力分数, 团队协作能力分数, 组织与管理能力分数
：
1. **专业知识与技能**
2. **学习能力**
3. **通识性知识与技能**
4. **沟通与表达能力**
5. **创新能力**
6. **执行能力**
7. **自我管理能力**
8. **职业适应能力**
9. **动手操作能力**
10. **承压抗挫能力**
11. **团队协作能力**
12. **组织与管理能力**

**职业**: "{adjusted_job}"  
**职位描述**: "{detail}"  
**学历要求**: "{edu_bg}"  
**经验要求**: "{expe}"

"""

    completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)
    return completion.choices[0].message.content


# 初始化空列表存储结果
results = []
results_abnor = []

jobs_unique = set(jobs_merged['adjusted_job'])


# 循环处理每条职位描述
for job_info in jobs_unique:
    job_title = job_info
    job_description = jobs_merged[jobs_merged['adjusted_job']==job_title]['detail']
    edu_background = jobs_merged[jobs_merged['adjusted_job']==job_title]['edu_bg']
    experience = jobs_merged[jobs_merged['adjusted_job']==job_title]['expe']
  
    result = extract_job_details(job_title, job_description, edu_background, experience)
    result_line = result.split(',')
    if(len(result_line)!=12):
        results_abnor.append({
            "工作岗位":job_title,
            "结果":result
        })
        continue

    major = result_line[0]
    study = result_line[1]
    genral_know = result_line[2]
    communi = result_line[3]
    innova = result_line[4]
    execute =result_line[5]
    self_manage = result_line[6]
    vocat_adjust = result_line[7]
    man_operat = result_line[8]
    pres_resis = result_line[9]
    team_opera = result_line[10]
    organ_manag = result_line[11]
    results.append({
    "工作岗位":job_title,
    "专业知识与技能分数":major,
    "学习能力分数":study,
    "通识性知识与技能分数":genral_know,
    "沟通与表达能力分数":communi,
    "创新能力分数":innova,
    "执行能力分数":execute,
    "自我管理能力分数":self_manage,
    "职业适应能力分数":vocat_adjust,
    "动手操作能力分数":man_operat,
    "承压抗挫能力分数":pres_resis,
    "团队协作能力分数":team_opera,
    "组织与管理能力分数":organ_manag,
    })

print(results)
print(results_abnor)

[{'工作岗位': '管培生', '专业知识与技能分数': '4', '学习能力分数': ' 4', '通识性知识与技能分数': ' 3', '沟通与表达能力分数': ' 5', '创新能力分数': ' 3', '执行能力分数': ' 4', '自我管理能力分数': ' 4', '职业适应能力分数': ' 4', '动手操作能力分数': ' 3', '承压抗挫能力分数': ' 4', '团队协作能力分数': ' 5', '组织与管理能力分数': ' 4'}, {'工作岗位': '市场营销', '专业知识与技能分数': '5', '学习能力分数': ' 4', '通识性知识与技能分数': ' 4', '沟通与表达能力分数': ' 5', '创新能力分数': ' 4', '执行能力分数': ' 5', '自我管理能力分数': ' 5', '职业适应能力分数': ' 4', '动手操作能力分数': ' 3', '承压抗挫能力分数': ' 4', '团队协作能力分数': ' 5', '组织与管理能力分数': ' 4'}, {'工作岗位': '供应链主管/经理', '专业知识与技能分数': '4', '学习能力分数': ' 4', '通识性知识与技能分数': ' 3', '沟通与表达能力分数': ' 4', '创新能力分数': ' 3', '执行能力分数': ' 5', '自我管理能力分数': ' 4', '职业适应能力分数': ' 4', '动手操作能力分数': ' 4', '承压抗挫能力分数': ' 4', '团队协作能力分数': ' 5', '组织与管理能力分数': ' 5'}, {'工作岗位': '行政助理/文员', '专业知识与技能分数': '4', '学习能力分数': ' 4', '通识性知识与技能分数': ' 3', '沟通与表达能力分数': ' 3', '创新能力分数': ' 2', '执行能力分数': ' 4', '自我管理能力分数': ' 4', '职业适应能力分数': ' 4', '动手操作能力分数': ' 3', '承压抗挫能力分数': ' 4', '团队协作能力分数': ' 4', '组织与管理能力分数': ' 3'}, {'工作岗位': '客服', '专业知识与技能分数': '3', '学习能力分数': ' 4', '通识性知识与技能分数': ' 

In [19]:
results_abnor = [
    {'工作岗位': '文科教师', '专业知识与技能分数': '5', '学习能力分数': '4', '通识性知识与技能分数': '5', '沟通与表达能力分数': '5', '创新能力分数': '3', '执行能力分数': '4', '自我管理能力分数': '4', '职业适应能力分数': '3', '动手操作能力分数': '2', '承压抗挫能力分数': '4', '团队协作能力分数': '5', '组织与管理能力分数': '3'},
    {'工作岗位': '项目管理', '专业知识与技能分数': '4', '学习能力分数': '3', '通识性知识与技能分数': '3', '沟通与表达能力分数': '4', '创新能力分数': '3', '执行能力分数': '4', '自我管理能力分数': '4', '职业适应能力分数': '4', '动手操作能力分数': '2', '承压抗挫能力分数': '4', '团队协作能力分数': '3', '组织与管理能力分数': '3'},
    {'工作岗位': '广告设计', '专业知识与技能分数': '4', '学习能力分数': '3', '通识性知识与技能分数': '2', '沟通与表达能力分数': '3', '创新能力分数': '4', '执行能力分数': '4', '自我管理能力分数': '3', '职业适应能力分数': '3', '动手操作能力分数': '2', '承压抗挫能力分数': '2', '团队协作能力分数': '3', '组织与管理能力分数': '2'},
    {'工作岗位': '人事专员', '专业知识与技能分数': '4', '学习能力分数': '4', '通识性知识与技能分数': '3', '沟通与表达能力分数': '4', '创新能力分数': '2', '执行能力分数': '4', '自我管理能力分数': '3', '职业适应能力分数': '3', '动手操作能力分数': '3', '承压抗挫能力分数': '4', '团队协作能力分数': '4', '组织与管理能力分数': '2'},
    {'工作岗位': '销售人员', '专业知识与技能分数': '4', '学习能力分数': '3', '通识性知识与技能分数': '3', '沟通与表达能力分数': '4', '创新能力分数': '2', '执行能力分数': '4', '自我管理能力分数': '3', '职业适应能力分数': '4', '动手操作能力分数': '3', '承压抗挫能力分数': '4', '团队协作能力分数': '4', '组织与管理能力分数': '3'}
]
results.extend(results_abnor)

In [20]:
df = pd.DataFrame(results)

In [21]:
df.to_csv('gpt3.5_turbo_mark.csv',encoding='utf-8')